In [ ]:
import os

# CHANGE WORKING DIRECTORY TO ROOT
current_dir = os.path.basename(os.getcwd())
if current_dir == "src":
    os.chdir("..") # Move up by 1
elif os.path.basename(os.getcwd()) == "bai-thesis-nlp":  
    pass # If already at root, stay there
else:
    os.chdir("../..") # Move up by 2 otherwise
    
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import re
from src._utils._helpers import log_synthetic_data, response2json, get_response, set_seed, clear_cuda_cache, get_generated_examples_df
from src._utils._run_multiclassRoBERTA import *
from sklearn.model_selection import train_test_split 

Error importing huggingface_hub.hf_api: No module named 'requests.exceptions'


ModuleNotFoundError: No module named 'requests.exceptions'

In [ ]:
real_train_df = pd.read_csv("real_data/train/agnewstrainAll.csv").rename(columns={"2": "text", "3": "label"})
real_train_df.drop(columns=["0", "1"], inplace=True)
real_train_df, dev_df = train_test_split(real_train_df, test_size=500, random_state=42, stratify=real_train_df["label"])

syn_rand_df, _ = get_generated_examples_df("synthetic_data/datasets/syn_agnews_baseline_500.json")
syn_targ_df, _ = get_generated_examples_df("synthetic_data/datasets/syn_agnews_baseline_500.json")

NameError: name 'pd' is not defined

In [ ]:
combined_df = combine_datasets(real_train_df, syn_rand_df, 0.5, 500)
display(combined_df.head())
print(combined_df.shape)

,text,label
0,An agricultural firm showcased its latest prod...,Sports
1,WASHINGTON - The Congressional Budget Office p...,World
2,"A sports event has been announced, with a focu...",Sports
3,Nortel Networks said on Thursday it will slash...,Sci/Tech
4,"HEMPSTEAD, NY - He was drafted in the first ro...",Sports


(500, 2)


In [ ]:
common_texts = combined_df['text'].isin(syn_rand_df['text'])
num_common_texts = common_texts.sum()
print(f"Number of common texts: {num_common_texts}")

Number of common texts: 250


In [ ]:
output_dir = "src/agnews/experiments/exp_random_aug_50"
os.makedirs(output_dir, exist_ok=True)
# exp_targeted_aug_30
# exp_real_only
# save datasets for reproducibility
train_path = os.path.join(output_dir, "train.csv")
dev_path = os.path.join(output_dir, "dev.csv")
combined_df.to_csv(train_path, index=False)
dev_df.to_csv(dev_path, index=False)

In [ ]:
# preprocess data
labels = combined_df["label"].unique().tolist()
label2id = {label: idx for idx, label in enumerate(labels)}
train_dataset = preprocess_data(Dataset.from_pandas(combined_df), label2id)
dev_dataset = preprocess_data(Dataset.from_pandas(dev_df), label2id)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
# train
start_time = time.time()
labels_ids = [label2id[label] for label in labels]
train_model(train_dataset, dev_dataset, labels_ids, output_dir)
end_time = time.time()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,F1 Macro
1,No log,0.798011,0.818000,0.818000,0.639034
2,No log,0.382300,0.874000,0.874000,0.796680
3,No log,0.344867,0.890000,0.890000,0.849163
4,No log,0.373569,0.890000,0.890000,0.830985
5,No log,0.355465,0.896000,0.896000,0.858061
6,No log,0.372408,0.906000,0.906000,0.868023
7,No log,0.411737,0.884000,0.884000,0.836690
8,No log,0.413848,0.886000,0.886000,0.841941


Training completed for experiment saved in: src/agnews/experiments/exp_random_aug_50
